# MIMIC 4 data - dataset construction labevents

Code taken from GRU-ODE-Bayes preprocessing; simplified and adapted for MIMIC 4 1.0

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [3]:
adm = pd.read_csv(r"D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/admissions_processed.csv")
adm.head()

,Unnamed: 0,subject_id,anchor_age,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,elapsed_time,elapsed_days
0,117,10002114,56,27793700,2162-02-17 22:32:00,2162-03-04 15:16:00,NaN,OBSERVATION ADMIT,P46834,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicaid,English,NaN,UNKNOWN,2162-02-17 19:35:00,2162-02-17 23:30:00,0,14 days 16:44:00,14
1,137,10002348,77,22725460,2112-11-30 22:22:00,2112-12-10 17:56:00,NaN,OBSERVATION ADMIT,P25W49,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,English,SINGLE,WHITE,2112-11-30 15:08:00,2112-11-30 23:24:00,0,9 days 19:34:00,9
2,151,10002495,81,24982426,2141-05-22 20:17:00,2141-05-29 17:41:00,NaN,URGENT,P336JM,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,English,MARRIED,UNKNOWN,NaN,NaN,0,6 days 21:24:00,6
3,163,10002667,58,23197839,2187-02-23 16:01:00,2187-02-28 16:00:00,NaN,OBSERVATION ADMIT,P95BQY,PHYSICIAN REFERRAL,HOME HEALTH CARE,Private,English,NaN,WHITE,NaN,NaN,0,4 days 23:59:00,4
4,166,10002760,56,28094813,2141-04-20 07:15:00,2141-04-24 13:31:00,NaN,SURGICAL SAME DAY ADMISSION,P32CZ5,PHYSICIAN REFERRAL,HOME HEALTH CARE,Private,English,NaN,UNABLE TO OBTAIN,NaN,NaN,0,4 days 06:16:00,4


In [4]:
df = pd.DataFrame()
for chunk in pd.read_csv(r"D:\Dataset\mimic-iv-3.0\mimic-iv-3.0\hosp/labevents.csv.gz", chunksize=500000):
    adm_ids=list(adm["hadm_id"])
    chunk=chunk.loc[chunk["hadm_id"].isin(adm_ids)]
    df = df.append(chunk[["subject_id","hadm_id","charttime","valuenum","itemid"]])

C:\Users\huzep\AppData\Local\Temp\ipykernel_23996\3368774938.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(chunk[["subject_id","hadm_id","charttime","valuenum","itemid"]])
C:\Users\huzep\AppData\Local\Temp\ipykernel_23996\3368774938.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(chunk[["subject_id","hadm_id","charttime","valuenum","itemid"]])
C:\Users\huzep\AppData\Local\Temp\ipykernel_23996\3368774938.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(chunk[["subject_id","hadm_id","charttime","valuenum","itemid"]])
C:\Users\huzep\AppData\Local\Temp\ipykernel_23996\3368774938.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

In [5]:
# only choose previously selected admission ids.
print("Number of patients remaining in the database: ")
print(df["subject_id"].nunique())

Number of patients remaining in the database: 
21976


In [6]:
# get item ids
item_id=pd.read_csv(r"D:\Dataset\mimic-iv-3.0\mimic-iv-3.0\hosp/d_labitems.csv.gz")
item_id_1=item_id[["itemid","label"]]
item_id_1.head()

,itemid,label
0,50801,Alveolar-arterial Gradient
1,50802,Base Excess
2,50803,"Calculated Bicarbonate, Whole Blood"
3,50804,Calculated Total CO2
4,50805,Carboxyhemoglobin


In [7]:
# get names of administered items
lab2=pd.merge(df,item_id_1,on="itemid")
lab2.head()
print("Number of patients remaining in the database: ")
print(lab2["subject_id"].nunique())

Number of patients remaining in the database: 
21976


In [8]:
lab2['itemid'].nunique()


811

In [9]:
# get only top 150 most used tests
n_best=150

n_best=1000
pat_for_item=lab2.groupby("label")["subject_id"].nunique()
frequent_labels=pat_for_item.sort_values(ascending=False)[:n_best]
lab3=lab2.loc[lab2["label"].isin(list(frequent_labels.index))].copy()

print("Number of patients remaining in the database: ")
print(lab3["subject_id"].nunique())

Number of patients remaining in the database: 
21976


In [10]:
lab3['label'].unique()
pd.Series(lab3['label'].unique(), name="UniqueNames").to_csv("unique_names.csv", index=False)

In [11]:
# only select the subset that was used in the paper (only missing is INR(PT))
subset=["Albumin","Alanine Aminotransferase (ALT)","Alkaline Phosphatase","Anion Gap","Asparate Aminotransferase (AST)","Base Excess","Basophils","Bicarbonate","Bilirubin, Total","Calcium, Total","Calculated Total CO2","Chloride","Creatinine","Eosinophils","Glucose","Hematocrit","Hemoglobin",
"Lactate","Lymphocytes","MCH","MCV","Magnesium","Monocytes","Neutrophils","PT","PTT","Phosphate","Platelet Count","Potassium","RDW","Red Blood Cells","Sodium","Specific Gravity","Urea Nitrogen","White Blood Cells","pCO2","pH","pO2"]
print(len(subset))
subset = ["White Blood Cells", "Absolute Lymphocyte Count", "Absolute Neutrophil Count", "Platelet Count", "Urea Nitrogen", "Potassium","Chloride",
          "Creatinine", "Sodium", "Albumin", "Bilirubin", "C-Reactive Protein", "Lactate", "pH", "pO2", "Bicarbonate", "Hematocrit"]
# subset += [""]

lab3_final=lab3.loc[lab3["label"].isin(subset)].copy()

38


In [12]:
lab3_final.to_csv(r"D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/lab_processed.csv")